This is the training of the BERT models. It returns a list of probabilities for each review used as input for the classification model

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 

In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Thesis DSS/Second_submission/review_helpful.csv', sep = ";")
data['review'] = data['review'].fillna('')
data['helpful'] = (data['helpful'] != 0).astype(int)

In [32]:
data['review'] = data['review'].str.lower()

In [4]:
data = pd.DataFrame({'text': data['review'], 'labels': data['helpful']})

In [15]:
data

,text,labels
0,This brand DVD player works great. Will be bu...,0
1,This DVD player upscales all our DVD's. Old D...,1
2,5 to 10 second delay in input from remote is a...,0
3,"Like I said a great picture, great value for t...",0
4,This compact DVD player easily fit atop our co...,0
...,...,...
121574,I went through 2 Samsung smart phones in the s...,0
121575,"I've only had it for a couple weeks, but it's ...",0
121576,Don't buy this if you are trying to activate i...,0
121577,"Can't complain about speed, it performs fine.C...",0


In [5]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)

In [46]:
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationArgs
model_args = ClassificationArgs(
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    max_seq_length=512,
    eval_batch_size=8
)
# Create a TransformerModel
#model = ClassificationModel("xlnet", "xlnet-base-cased", args=model_args, use_cuda=True)
#model = ClassificationModel('bert', 'bert-base-uncased', args=model_args, use_cuda=True)
model = ClassificationModel('roberta', 'roberta-large', args=model_args, use_cuda = True)
#model = ClassificationModel('roberta', 'roberta-base', args=model_args, use_cuda=True)
#model = ClassificationModel('bert', 'bert-large-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('distilbert', 'distilbert-base-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('albert', 'albert-base-v2', args=model_args, use_cuda=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
import gc
import torch

del model
gc.collect()

torch.cuda.empty_cache()

# Optionally, you can print GPU memory usage to verify
print(f"Memory Allocated: {torch.cuda.memory_allocated()} bytes")
print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated()} bytes")
print(f"Memory Cached: {torch.cuda.memory_reserved()} bytes")
print(f"Max Memory Cached: {torch.cuda.max_memory_reserved()} bytes")


Memory Allocated: 1686079488 bytes
Max Memory Allocated: 23517254144 bytes
Memory Cached: 5012193280 bytes
Max Memory Cached: 23538434048 bytes


In [47]:
history = model.train_model(train_df, args={'num_train_epochs': 2, 'learning_rate': 3e-5, 'overwrite_output_dir': True,  'train_batch_size' : 8})

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/9119 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


KeyboardInterrupt: 

In [30]:
result_train, model_outputs_train, wrong_predictions_train = model.eval_model(train_df)
result_val, model_outputs_val, wrong_predictions_val = model.eval_model(val_df)

Running Evaluation:   0%|          | 0/2280 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/760 [00:00<?, ?it/s]

# Get model performance

In [19]:
#results = pd.DataFrame(result_val, index = [0])
results['train_loss'] = result_train['eval_loss']
results['name'] = "Roberta large"
results['train_accuracy'] = result_train['accuracy']
results['seq_length'] = 64
#results.to_csv('/content/drive/MyDrive/Thesis DSS/Second_submission/sequence_comparison.csv', sep = ";", index=False)

In [31]:
results_new = pd.DataFrame(result_val, index = [0])
results_new['train_loss'] = result_train['eval_loss']
results_new['train_accuracy'] = result_train['accuracy']
results_new['name'] = "Roberta large"
results_new['seq_length'] = 256
results = pd.read_csv('/content/drive/MyDrive/Thesis DSS/Second_submission/sequence_comparison.csv', sep=';')
results = pd.concat([results, results_new])
results.reset_index(drop = True, inplace = True)
results.to_csv('/content/drive/MyDrive/Thesis DSS/Second_submission/sequence_comparison.csv', sep = ";", index = False)

In [ ]:
results = pd.read_csv('/content/drive/MyDrive/Thesis DSS/Second_submission/model_comparison.csv', sep=';', index_col=False)
results

,mcc,accuracy,f1_score,tp,tn,fp,fn,auroc,auprc,eval_loss,train_loss,name,train_accuracy,seq_length
0,0.366158,0.685228,0.676688,6355,10307,2675,4979,0.745223,0.719034,0.594911,0.569546,xlnet-base-cased,0.704251,128
1,0.367153,0.686256,0.681624,6877,9810,3172,4457,0.743357,0.716974,0.596254,0.549756,bert-base-uncased,0.721839,128


In [ ]:
results.to_csv('roberta_seq_length.csv', sep = ";")

In [ ]:
model.save_model("./roberto")